In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Streamming").getOrCreate()

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ee1715b1-5277-4789-b871-72b99b373bf5;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 303ms :: artifacts dl 7ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [3]:
from pyspark.sql.types import StructType

userSchema = StructType().add("article", "string").add("rank", "double")
df = spark.readStream \
    .option("sep", "\t") \
    .schema(userSchema) \
    .csv("hdfs://hw2-m/user/root/p2t1")

In [4]:
df = df.select('article', 'rank').where("rank > 0.5")

In [5]:
df.writeStream \
    .format("csv") \
    .option("delimiter", "\t") \
    .option("checkpointLocation", "hdfs://hw2-m/user/root/p2t2") \
    .option("path", "hdfs://hw2-m/user/root/p2t2") \
    .start()

22/04/21 20:50:06 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
